<a href="https://colab.research.google.com/github/RodericGuigoCorominas/datascience/blob/main/build_your_own_net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
###DO NOT EDIT THIS CODE
################################################################################################################################
from google.colab import drive
drive.mount('/content/gdrive/')
%cd '/content/gdrive/MyDrive/Easy Nets/'
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torchvision import datasets, transforms
from PIL import Image
import matplotlib.pyplot as plt

#GPU or CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#Loss Function
loss_function = nn.CrossEntropyLoss()

#Training Function
def train(epochs, model, dataloader, testloader, optimizer, loss_function):
  for epoch in range(epochs):
    loss_epoch = np.array([])
    correct, total = (0,0)
    for data, labels in dataloader:
      input_data = Variable(data).to(device)
      labels = labels.to(device)
      predict = model(input_data)
      loss = loss_function(predict, labels)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      loss_epoch = np.append(loss_epoch, loss.item())
    
    for data, labels in testloader:
      input_data = Variable(data).to(device)
      labels = labels.to(device)
      predict = model(input_data)
      for i, out in enumerate(predict):
        pred = torch.argmax(out)
        if pred == labels[i]:
          correct+=1
        total+=1

    accuracy = correct/total    
  
    print('epoch [{}/{}], training loss:{:.4f}, accuracy:{:.4f}'.format(epoch+1, epochs, np.mean(loss_epoch), accuracy))
################################################################################################################################

def num_params(model):
  return sum([p.numel() for p in model.parameters()])

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).
/content/gdrive/.shortcut-targets-by-id/1IrVCGk9CH8ZYDWEH4w96PenFpHYJx6Cs/Easy Nets


# **Load Dataset**

Available datasets are: MNIST, CIFAR10

## MNIST

In [ ]:
# download and load data
batch_size = 512

# download and transform train dataset
train_loader = torch.utils.data.DataLoader(datasets.MNIST('./mnist_data', download=True, train=True, transform=transforms.Compose([
                                                transforms.ToTensor(), # first, convert image to PyTorch tensor
                                                transforms.Normalize((0.1307,), (0.3081,)) # normalize inputs
                                                ])), batch_size=batch_size, shuffle=True)

# download and transform test dataset
test_loader = torch.utils.data.DataLoader(datasets.MNIST('./mnist_data', download=True, train=False, transform=transforms.Compose([
                                                              transforms.ToTensor(), # first, convert image to PyTorch tensor
                                                              transforms.Normalize((0.1307,), (0.3081,)) # normalize inputs
                                                          ])), batch_size=batch_size, shuffle=False)

## SVHN

In [ ]:
# download and load data
batch_size = 512

# download and transform train dataset
train_dataset = datasets.SVHN(root='/content/gdrive/.shortcut-targets-by-id/1IrVCGk9CH8ZYDWEH4w96PenFpHYJx6Cs/Easy Nets', 
                                                         split='train', transform=None, download=True)
train_dataset = [(transforms.ToTensor()(img[0]), img[1]) for img in train_dataset]
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# download and transform test dataset
test_dataset = datasets.SVHN(root='/content/gdrive/.shortcut-targets-by-id/1IrVCGk9CH8ZYDWEH4w96PenFpHYJx6Cs/Easy Nets',
                                                        split='test', transform=None, download=True)
test_dataset = [(transforms.ToTensor()(img[0]), img[1]) for img in test_dataset]
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

Using downloaded and verified file: /content/gdrive/.shortcut-targets-by-id/1IrVCGk9CH8ZYDWEH4w96PenFpHYJx6Cs/Easy Nets/train_32x32.mat
Using downloaded and verified file: /content/gdrive/.shortcut-targets-by-id/1IrVCGk9CH8ZYDWEH4w96PenFpHYJx6Cs/Easy Nets/test_32x32.mat


In [ ]:
### Test Dimensions
iterable = iter(train_loader)
print(iterable.next()[0].shape)
print(iterable.next()[1].shape)

torch.Size([512, 3, 32, 32])
torch.Size([512])


# **Build Network**

In [ ]:
# neural network
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    ###Add Layers

  def forward(self, x):
    ###Feed Forward
    return(x)

In [ ]:
# create model object
model = CNN().to(device)

# **Define Hyperparameters**

In [ ]:
learning_rate = 10e-3
weight_decay = 10e-5
n_epochs = 10

# **Chose Optimizer**

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
#optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

# **Train and Validate**

In [ ]:
train(n_epochs, model, train_loader, test_loader, optimizer, loss_function)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


epoch [1/10], training loss:1.3661, accuracy:0.8138
epoch [2/10], training loss:0.4834, accuracy:0.8557
epoch [3/10], training loss:0.3918, accuracy:0.8768
epoch [4/10], training loss:0.3531, accuracy:0.8861
epoch [5/10], training loss:0.3224, accuracy:0.8871
epoch [6/10], training loss:0.3049, accuracy:0.8928
epoch [7/10], training loss:0.2869, accuracy:0.9020
epoch [8/10], training loss:0.2736, accuracy:0.8940
epoch [9/10], training loss:0.2630, accuracy:0.9011
epoch [10/10], training loss:0.2583, accuracy:0.9098


# **No Batch Normalization**

In [ ]:
# neural network
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.conv1 = nn.Conv2d(1, 16, kernel_size=5, padding=2)
    self.conv2 = nn.Conv2d(16, 16, kernel_size=3, padding=1)
    self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
    self.conv3 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
    self.conv4 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
    self.conv5 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
    self.avg = nn.AdaptiveAvgPool2d(4)
    self.fc = nn.Linear(1024,10)
    self.softmax = nn.LogSoftmax(dim=1)

  def forward(self, x):
    x = torch.relu(self.conv1(x))
    x = torch.relu(self.conv2(x))
    x = self.maxpool(x)
    x = torch.relu(self.conv3(x))
    x = torch.relu(self.conv4(x))
    x = torch.relu(self.conv5(x))
    x = self.maxpool(x)
    x = self.avg(x)
    x = x.view(-1,1024)
    x = self.fc(x)
    x = self.softmax(x)
    return(x)

model = CNN().to(device)

# **Best Model**

In [ ]:
# neural network
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.conv1 = nn.Conv2d(3, 16, kernel_size=5, padding=2)
    self.conv2 = nn.Conv2d(16, 16, kernel_size=3, padding=1)
    self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
    self.bn1 = nn.BatchNorm2d(16)
    self.conv3 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
    self.conv4 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
    self.conv5 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
    self.bn2 = nn.BatchNorm2d(64)
    self.avg = nn.AdaptiveAvgPool2d(4)
    self.fc = nn.Linear(1024,10)
    self.softmax = nn.LogSoftmax(dim=1)

  def forward(self, x):
    x = torch.relu(self.conv1(x))
    x = torch.relu(self.conv2(x))
    x = self.maxpool(x)
    x = self.bn1(x)
    x = torch.relu(self.conv3(x))
    x = torch.relu(self.conv4(x))
    x = torch.relu(self.conv5(x))
    x = self.maxpool(x)
    x = self.bn2(x)
    x = self.avg(x)
    x = x.view(-1,1024)
    x = self.fc(x)
    x = self.softmax(x)
    return(x)

model = CNN().to(device)